In [ ]:
import sys
sys.path.append('../src/') 
import SEE_TC as ctc

import os
import tifffile
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedShuffleSplit

##### define GPU indices to use
GPU_2_use = 1 ##### define GPU indices to use
GPU_ids = [str(GPU_2_use)]  # Specify the GPUs you want to use
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU_2_use)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set for GPU(s).")
    except RuntimeError as e:
        print(f"Error setting memory growth: {e}")

In [ ]:
data_dir_train = "../train_data/"
img_cells = tifffile.imread(data_dir_train+"03_train_cell_isolator_single_cells.tiff")
img_clusters = tifffile.imread(data_dir_train+"03_train_cell_isolator_cell_clusters.tiff")
img_debris = tifffile.imread(data_dir_train+"03_train_cell_isolator_non_cellular_artifacts.tiff")
img_cells.shape, img_clusters.shape, img_debris.shape

In [ ]:
# Create labels
labels_cells = np.full((img_cells.shape[0],), 0)  # Class 0 for "A"
labels_clusters = np.full((img_clusters.shape[0],), 1)   # Class 1 for "B"
labels_debris = np.full((img_debris.shape[0],), 2)   # Class 3 for "D"

# Combine datasets and labels
X = np.concatenate((img_cells, img_clusters, img_debris), axis=0).transpose((0,2,3,1))
y = np.concatenate((labels_cells, labels_clusters, labels_debris), axis=0)

tiles_hoechst = ctc.preprocess_autoEncoder_01(X[:,:,:,0])
tiles_bins = X[:,:,:,1]
tiles_bins[tiles_bins > 0] = 255
tiles_bins = np.expand_dims(tiles_bins, axis=-1)  
X = np.concatenate((tiles_bins, np.expand_dims(tiles_hoechst,-1)), axis = 3) # merge all channels with binary

In [ ]:
# Shuffle and split the data into training and testing sets. Stratified split.
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2)

for train_index, test_index in split.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
input_shape = (32, 32, X.shape[3])
encoder_model = ctc.create_encoder_model(input_shape)

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100, mode = 'max', restore_best_weights= True)]
history = encoder_model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks)
test_loss, test_acc = encoder_model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

In [ ]:
from sklearn.metrics import confusion_matrix
# Make predictions on the test data
y_pred = encoder_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test, y_pred_classes) # Compute the confusion matrix
class_names = ["Cell", "Fused", "Debris"] # Class names
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # Normalize the confusion matrix by row
ctc.plot_normalized_confusion_matrix(cm, class_names) # Plot the normalized confusion matrix
